# RAG 

#### Libraries

In [481]:
# processing:
import os
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity

# neo4j:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# llm:
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retrieval:
from langchain.chains import RetrievalQAWithSourcesChain

# telegram:
import telebot
from time import sleep

#### .env

In [482]:
load_dotenv()

True

In [483]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
GIGACHAT_SCOPE = os.getenv('SCOPE')
GIGACHAT_AUTH = os.getenv('AUTH_DATA')

# telegram:
BOT_TOKEN = os.getenv('BOT_TOKEN')

#### Reference

In [484]:
reference = pd.read_excel('../docs/reference.xlsx')

## Setting Up Database

In [485]:
embeddings = GigaChatEmbeddings(credentials = GIGACHAT_AUTH, verify_ssl_certs = False)

In [486]:
vectorestore = Neo4jVector.from_existing_index(embeddings,
                                               url = NEO4J_URI,
                                               username = NEO4J_USERNAME,
                                               password = NEO4J_PASSWORD,
                                               index_name = "Document",
                                               node_label = "Document" ,
                                               keyword_index_name = "keywords",
                                               search_type = "hybrid")

In [487]:
graph = Neo4jGraph(url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE)

## Setting Up LLM


In [488]:
llm = GigaChat(model = "GigaChat-Pro", # "GigaChat-Pro" / "GigaChat-Plus" (32k)
               temperature = 0.3,
               top_p = 0.2,
               n = 1,
               repetition_penalty = 1,
               credentials = GIGACHAT_AUTH, 
               verify_ssl_certs = False)

In [489]:
parser = StrOutputParser()

## Prompt Engineering

#### Prompt Template

In [490]:
template = """
            Задача: Предоставлять детализированные ответы, используя указанные документы.

            Цитируйте конкретные пункты и статьи документов при ответе на вопросы, связанные с правилами.
            Объясните применение документов. Не повторяйте информацию и формулируйте ответы грамотно.

            Запрос: "{question}"
            Документы: "{context}"
            
            Ответ: Ответ должен содержать не более 200 слов и включать финальный вывод.
            """

prompt = ChatPromptTemplate.from_template(template)

#### Question Extraction

In [491]:
def extract_question(text):
    
    chain = llm | parser

    template = f"""
            Задача: Идентифицировать вопросы в предоставленном тексте.

            Текст запроса: "{text}"
            
            Анализируйте текст и выделяйте явные вопросы. Если в тексте нет вопросов, сформулируйте их на основе имеющейся информации.
            Результаты представьте как список вопросов, разделенных запятыми. Стремитесь к ясности и краткости.
                
            Ответ должен быть конкретным и содержать все ключевые аспекты, без дополнительных комментариев.
            """
    
    return chain.invoke(template).replace('/', ' ')

### Utilities

In [492]:
def find_relevant_bullets(inquiry):
    match = re.search(r'(\d+\.\d+)(?:\.\d+)?', inquiry)
    
    if match:
        number = match.group(1)
        
        cypher = f"""
                    MATCH (n:Document)
                    WHERE n.bullet STARTS WITH '{number}'
                    RETURN n.text
                    """
    
        mentioned_bullets = graph.query(cypher)
        
        if mentioned_bullets:
            texts = [bullet['n.text'] for bullet in mentioned_bullets]
            
            all_embeddings = [embeddings.embed_query(text) for text in texts] + [embeddings.embed_query(inquiry)]
            inquiry_embedding = all_embeddings[-1]
            bullet_embeddings = all_embeddings[:-1]
            
            similarities = cosine_similarity([inquiry_embedding], bullet_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:3]
            top_texts = [texts[i] for i in top_indices]
            
            return top_texts
        else:
            return "Отсутствуют"
    else:
        return "Отсутствуют"

## LLM Retrival Chain

#### Chain

In [493]:
retriever = vectorestore.as_retriever(search_kwargs={"k": 5, 'score_threshold': 0.9, 'lambda_mult': 0.25})

In [494]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm, # pro / plus
                                                    chain_type = "stuff", # "stuff" / "map_rerank" / "refine"
                                                    retriever = retriever,
                                                    return_source_documents = True,
                                                    reduce_k_below_max_tokens = False,
                                                    max_tokens_limit = 8000, # up to 32k with plus model
                                                    chain_type_kwargs = {"verbose": False,
                                                                         "prompt": prompt,
                                                                         "document_variable_name": "context"})

##### Outputs

In [495]:
letter = 19

inquiry = str(reference['letter'][letter-1].replace('\n', ' '))
question = extract_question(inquiry)

response = chain.invoke({"question": extract_question(inquiry)}, return_only_outputs = False)
response['answer']

'Для ответа на вопросы необходимо обратиться к указанным документам.\n\n1. Для договора, предполагающего работу по гарантированному и негарантированному объему, необходимо указать категорию "Рамочный договор".\n2. В приложении 1 не требуется добавлять формулировку о переводе негарантированного объема в гарантированный, так как это не предусмотрено указанными документами.\n3. Для пункта 5.4 рамочного договора рекомендуется использовать формулировку, определяющую единичные расценки и максимальную цену договора, а также указывающую на то, что номенклатура, объем, сроки и место поставки товаров (выполнения работ, оказания услуг) могут определяться заявками заказчика или иным образом, предусмотренным рамочным договором при возникновении потребности в товарах (работах, услугах).\n4. Изменения в формулировке пункта 5.4 для договора, прошедшего закупочную процедуру и согласование закупки, могут быть критичными, так как они могут повлиять на условия договора и привести к его недействительности.

In [496]:
source_documents = [doc.page_content for doc in response['source_documents']]
source_documents

['5.11. При закупках товаров (работ, услуг), когда невозможно заранее определить точный объем, конкретные сроки, место поставки товаров (выполнения работ, оказания услуг), но могут быть определены единичные расценки (прейскурант) на такие товары (работы, услуги), а также предельная стоимость и/или предельный объем закупки, Заказчиком может быть применен рамочный или прейскурантный договор. При этом условиями рамочного или прейскурантного договора определяются единичные расценки (прейскурант) и максимальная цена договора, иные условия договора, а номенклатура, объем, сроки и место поставки товаров (выполнения работ, оказания услуг) могут определяться заявками Заказчика или иным образом, предусмотренным рамочным или прейскурантным договором при возникновении у Заказчика потребности в товарах (работах, услугах).',
 'использования в описании предмета закупки указания на товарный знак необходимо использовать слова «(или эквивалент)», за исключением случаев: 1) несовместимости товаров, на ко

In [497]:
bullets = [doc.metadata['bullet'] for doc in response['source_documents']]
bullets

['5.11', '7.2.1.3', '4.5.10', '4.5.3.2', '5.3.4']

In [498]:
mentioned_bullets = find_relevant_bullets(inquiry)
mentioned_bullets

['5.4.1. При НМЦ планируемого к заключению договора менее 10 млн руб. без НДС: для Обществ Компания 1: решение о закупке у единственного поставщика (подрядчика, исполнителя) принимается Руководителем Общества Компания 1 (На время отсутствия Руководителя Общества Компания 1, полномочия по принятию решения могут быть возложены на лицо, исполняющее его обязанности на основании соответствующего приказа, при этом делегирование полномочий не предусмотрено. ) . для подразделений КЦ: решение о закупке у единственного поставщика (подрядчика, исполнителя) принимается Заместителем Генерального директора по профилю деятельности или Руководителем структурного подразделения прямого подчинения Генеральному директору ПАО «Компания 1» (На время отсутствия ЗГД по профилю деятельности / Руководителя СП ПП ГД ПАО «Компания 1» полномочия по принятию решения могут быть возложены на лицо, исполняющее его обязанности на основании соответствующего приказа, при этом делегирование полномочий не предусмотрено. ) 

## Test

#### Telegram

In [499]:
user_conversations = {}
bot = telebot.TeleBot(BOT_TOKEN)

In [500]:
@bot.message_handler(content_types=['audio', 'video', 'document', 'photo', 'sticker', 'voice', 'location', 'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')

@bot.message_handler(content_types=['text'])

def handle_text_message(message):
    user_id = message.chat.id
    extracted_question = extract_question(message.text)

    response = chain.invoke({"question": extracted_question}, return_only_outputs = False)

    bot.send_message(user_id, str(response['answer']))
    sleep(2)

In [501]:
bot.polling(none_stop=True)

#### Responses

In [ ]:
responses = pd.DataFrame(columns = ['letter', 'extracted questions', 'answer', 'rag answer', 'sources', 'mentioned bullets'])

for letter in range(1, 21):

    inquiry = str(reference['letter'][letter-1].replace('\n', ' ')) # letters from excel
    refer = str(reference['answer '][letter-1]).replace('\n', ' ') # reference answer from excel

    question = extract_question(inquiry)
    response = chain.invoke({"question": question}, return_only_outputs = False)
    rag_answer = response['answer']
    sources = [doc.metadata['bullet'] for doc in response['source_documents']]
    mentioned_bullets= str(find_relevant_bullets(inquiry)).replace('/', ' ')

    responses = pd.concat([responses, pd.DataFrame({'letter': [inquiry], 
                                                    'extracted questions': [question], 
                                                    'answer': [refer], 
                                                    'rag answer': [rag_answer], 
                                                    'sources': [sources],
                                                    'mentioned bullets': [mentioned_bullets]})
                                                    ], ignore_index = True)

In [ ]:
responses.head(3)

,letter,extracted questions,answer,rag answer,sources,mentioned documents
0,"Уважаемые коллеги, Добрый день! У меня возник...",1. Допустимо ли при расчете НМЦ указывать орие...,"Согласно пункту 5.11 ПоЗ, при закупках товаров...","1. В соответствии с документами, при расчете Н...","[3.1, 5.11, 4.5.13, 4.5.16, 4.5.16]",Отсутствуют
1,"Уважаемые коллеги, Обращаюсь с вопросом: При ...",1. При осуществлении закупки у единственного п...,Согласно СК-03.03.02.01 Проведение закупки у е...,При осуществлении закупки у единственного пост...,"[1.6, 5.3.6, 1.1, 4.1, 17.2]",Отсутствуют
2,"Уважаемые коллеги, У меня вопрос, связанный с...",1. Может ли быть заключен договор с участником...,"пункт 17.1.8. в случае, когда на участие в кон...","1. Да, договор может быть заключен с участнико...","[13.2.3, 14.5.1, 10.5.1, 14.12.6, 14.2.3]",Отсутствуют


In [ ]:
responses.to_excel('rag_test.xlsx', index=False)

#### Basic Queries

[Cypher](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise)

In [ ]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING},Document {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING},Test {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING}
Relationship properties are the following:

The relationships are the following:



In [ ]:
cypher = """
         SHOW VECTOR INDEXES
         """
graph.query(cypher)

In [ ]:
cypher = """
         MATCH (n)
         RETURN count(n)
         """
graph.query(cypher)

[{'count(n)': 2331}]

In [ ]:
cypher = """
         MATCH (n:Chunk {bullet: "5.4"})
         RETURN n.text AS text
         """
graph.query(cypher)

[{'text': '5.4. Уровень принятия решения о закупке у единственного поставщика (подрядчика, исполнителя)'},
 {'text': '5.4. При проведении закупок Организатор вправе привлекать подразделения Организатора (ПАО «Компания 1», Общества Компания 1 соответственно), Заказчика, обладающие необходимой компетенцией по предмету закупки.'},
 {'text': '5.4. Периоды формирования и корректировок ГПЗ Этап периода : Формирование ГПЗ на планируемый календарный год . Начало периода : 01 октября года, предшествующего планируемому  Завершение периода: до 31 декабря года, предшествующего планируемому включительно  Этап периода : Корректировка\xa0 утвержденного ГПЗ на текущий календарный год  Начало периода : 01 января  Завершение периода: до 31 декабря  года планирования включительно'}]